# Lab | LangChain Med

## Objectives

- continue on with lesson 2' example, use different datasets to test what we did in class. Some datasets are suggested in the notebook but feel free to scout other datasets on HuggingFace or Kaggle.
- Find another model on Hugging Face and compare it.
- Modify the prompt to fit your selected dataset.

In [1]:
import numpy as np 
import pandas as pd

## Load the Dataset
As you can see the notebook is ready to work with three different Datasets. Just uncomment the lines of the Dataset you want to use. 

I selected Datasets with News. Two of them have just a brief decription of the news, but the other contains the full text. 

As we are working in a free and limited space, I limited the number of news to use with the variable MAX_NEWS. Feel free to pull more if you have memory available. 

The name of the field containing the text of the new is stored in the variable *DOCUMENT* and the metadata in *TOPIC*

In [ ]:
news = pd.read_csv('labelled_newscatcher_dataset.csv', sep=';')
MAX_NEWS = 1000
DOCUMENT = "title"
TOPIC = "topic"

'''
news = pd.read_csv('bbc_news.csv')
MAX_NEWS = 1000
DOCUMENT = "description"
TOPIC = "title"
'''


'\nnews = pd.read_csv(\'bbc_news.csv\')\nMAX_NEWS = 1000\nDOCUMENT = "description"\nTOPIC = "title"\n'

ChromaDB requires that the data has a unique identifier. We can make it with this statement, which will create a new column called **Id**.


In [5]:
news["id"] = news.index
news.head()

,topic,link,domain,published_date,title,lang,id
0,SCIENCE,https://www.eurekalert.org/pub_releases/2020-0...,eurekalert.org,2020-08-06 13:59:45,A closer look at water-splitting's solar fuel ...,en,0
1,SCIENCE,https://www.pulse.ng/news/world/an-irresistibl...,pulse.ng,2020-08-12 15:14:19,"An irresistible scent makes locusts swarm, stu...",en,1
2,SCIENCE,https://www.express.co.uk/news/science/1322607...,express.co.uk,2020-08-13 21:01:00,Artificial intelligence warning: AI will know ...,en,2
3,SCIENCE,https://www.ndtv.com/world-news/glaciers-could...,ndtv.com,2020-08-03 22:18:26,Glaciers Could Have Sculpted Mars Valleys: Study,en,3
4,SCIENCE,https://www.thesun.ie/tech/5742187/perseid-met...,thesun.ie,2020-08-12 19:54:36,Perseid meteor shower 2020: What time and how ...,en,4


In [ ]:
subset_news = news.head(MAX_NEWS)

In [7]:
display(subset_news)

,topic,link,domain,published_date,title,lang,id
0,SCIENCE,https://www.eurekalert.org/pub_releases/2020-0...,eurekalert.org,2020-08-06 13:59:45,A closer look at water-splitting's solar fuel ...,en,0
1,SCIENCE,https://www.pulse.ng/news/world/an-irresistibl...,pulse.ng,2020-08-12 15:14:19,"An irresistible scent makes locusts swarm, stu...",en,1
2,SCIENCE,https://www.express.co.uk/news/science/1322607...,express.co.uk,2020-08-13 21:01:00,Artificial intelligence warning: AI will know ...,en,2
3,SCIENCE,https://www.ndtv.com/world-news/glaciers-could...,ndtv.com,2020-08-03 22:18:26,Glaciers Could Have Sculpted Mars Valleys: Study,en,3
4,SCIENCE,https://www.thesun.ie/tech/5742187/perseid-met...,thesun.ie,2020-08-12 19:54:36,Perseid meteor shower 2020: What time and how ...,en,4
...,...,...,...,...,...,...,...
995,TECHNOLOGY,https://www.androidcentral.com/mate-40-will-be...,androidcentral.com,2020-08-07 17:12:33,The Mate 40 will be the last Huawei phone with...,en,995
996,SCIENCE,https://www.cnn.com/2020/08/17/africa/stone-ag...,cnn.com,2020-08-17 17:10:00,"Early humans knew how to make comfy, pest-free...",en,996
997,HEALTH,https://www.tenterfieldstar.com.au/story/68776...,tenterfieldstar.com.au,2020-08-13 03:26:06,Regional Vic set for virus testing blitz,en,997
998,HEALTH,https://news.sky.com/story/coronavirus-trials-...,news.sky.com,2020-08-13 13:22:58,Coronavirus: Trials of second contact-tracing ...,en,998


## Import and configure the Vector Database
I'm going to use ChromaDB, the most popular OpenSource embedding Database. 

First we need to import ChromaDB, and after that import the **Settings** class from **chromadb.config** module. This class allows us to change the setting for the ChromaDB system, and customize its behavior. 

In [8]:
import chromadb
from chromadb.config import Settings

Now we need to create the seetings object calling the **Settings** function imported previously. We store the object in the variable **settings_chroma**.

Is necessary to inform two parameters 
* chroma_db_impl. Here we specify the database implementation and the format how store the data. I choose ***duckdb***, because his high-performace. It operate primarly in memory. And is fully compatible with SQL. The store format ***parquet*** is good for tabular data. With good compression rates and performance. 

* persist_directory: It just contains the directory where the data will be stored. Is possible work without a directory and the data will be stored in memory without persistece, but Kaggle dosn't support that. 

In [9]:
chroma_client = chromadb.PersistentClient(path = "./") # creates chroma.sqlite3

## Filling and Querying the ChromaDB Database
The Data in ChromaDB is stored in collections. If the collection exist we need to delete it. 

In the next lines, we are creating the collection by calling the ***create_collection*** function in the ***chroma_client*** created above.

In [10]:
print([chroma_client.list_collections()])

[['news_collection']]


In [11]:
collection_name = "news_collection"

if len(chroma_client.list_collections()) > 0 and collection_name in chroma_client.list_collections():
        chroma_client.delete_collection(name = collection_name)

collection = chroma_client.create_collection(name = collection_name)
    

In [12]:
print(collection, type(collection))

print(chroma_client, type(chroma_client))


Collection(name=news_collection) <class 'chromadb.api.models.Collection.Collection'>
<chromadb.api.client.Client object at 0x7fb371fc6330> <class 'chromadb.api.client.Client'>


It's time to add the data to the collection. Using the function ***add*** we need to inform, at least ***documents***, ***metadatas*** and ***ids***. 
* In the **document** we store the big text, it's a different column in each Dataset. 
* In **metadatas**, we can informa a list of topics. 
* In **id** we need to inform an unique identificator for each row. It MUST be unique! I'm creating the ID using the range of MAX_NEWS. 


In [13]:

collection.add(
    documents = subset_news[DOCUMENT].tolist(),
    metadatas = [{TOPIC: topic} for topic in subset_news[TOPIC].tolist()],
    ids = [f"id{x}" for x in range(MAX_NEWS)],
)

In [14]:
results = collection.query(query_texts = ["laptop"], n_results = 10 )

display(results)

'''
{'ids': [['id775', 'id707', 'id310', 'id587', 'id444', 'id751', 'id701', 'id862', 'id191', 'id740']],
 
 'embeddings': None,
 
 'documents': [['Photography student Thorsten Mjölnir captures the way students decorate their laptops.',
                'Why sales of very basic mobile phones, without apps and internet connection, are increasing.',
                "What do you do when your collection of millions of books keeps growing but your bookshelves don't?",
                'The developers of a powerful mini aircraft hope it will be used by the armed forces.',
                'How tech is helping young families and couples regain their busy social lives after Covid.',
                'Watch as Lee Zii Jia of Malaysia records a speed of 372km/h on his backhand point against Lakshya Sen of India in the All England Badminton Championships.',
                'The Royal Mint has found a way to turn old circuit boards from phones, computers and TVs into gold.',
                'A van was reportedly hijacked and driven to the venue, and a controlled explosion has since been carried out.',
                'The Ukrainian president reveals his location in Kyiv in a new video shared on social media.',
                'The Royal Mint has found a way to turn old circuit boards from phones, computers and televisions into gold.']],
 'uris': None,
 'data': None,

 'metadatas': [[{'title': 'Laptop art: From Vans to Harry Styles'},
                {'title': "Not smart but clever? The return of 'dumbphones'"},
                {'title': 'Building a bigger home for the British Library collection'},
                {'title': 'How a jetpack design helped create a flying motorbike'},
                {'title': 'The popular apps aiming to tame the chaos of family life'},
                {'title': "All England Badminton Championships: Malaysia's Lee Zii Jia records 372km/h backhand"},
                {'title': 'E-waste: Royal Mint wants to turn your old phone into gold'},
                {'title': "Simon Coveney's visit to Belfast interrupted by security alert"},
                {'title': 'War in Ukraine: President Zelensky posts video from office in Kyiv'},
                {'title': 'E-waste: Royal Mint wants to turn your old phone into gold'}]],

 'distances': [[1.106649398803711,
                1.630934715270996,
                1.6354169845581055,
                1.651792049407959,
                1.655225157737732,
                1.6666067838668823,
                1.6682372093200684,
                1.6726526021957397,
                1.679142951965332,
                1.6888483762741089]],

 'included': [<IncludeEnum.distances: 'distances'>,
              
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}
  '''

{'ids': [['id173',
   'id829',
   'id117',
   'id535',
   'id141',
   'id218',
   'id390',
   'id273',
   'id56',
   'id900']],
 'embeddings': None,
 'documents': [['The Legendary Toshiba is Officially Done With Making Laptops',
   '3 gaming laptop deals you can’t afford to miss today',
   'Lenovo and HP control half of the global laptop market',
   'Asus ROG Zephyrus G14 gaming laptop announced in India',
   'Acer Swift 3 featuring a 10th-generation Intel Ice Lake CPU, 2K screen, and more launched in India for INR 64999 (US$865)',
   "Apple's Next MacBook Could Be the Cheapest in Company's History",
   "Features of Huawei's Desktop Computer Revealed",
   'Redmi to launch its first gaming laptop on August 14: Here are all the details',
   'Toshiba shuts the lid on laptops after 35 years',
   'This is the cheapest Windows PC by a mile and it even has a spare SSD slot']],
 'uris': None,
 'data': None,
 'metadatas': [[{'topic': 'TECHNOLOGY'},
   {'topic': 'TECHNOLOGY'},
   {'topic': 'TECH

'\n{\'ids\': [[\'id775\', \'id707\', \'id310\', \'id587\', \'id444\', \'id751\', \'id701\', \'id862\', \'id191\', \'id740\']],\n \n \'embeddings\': None,\n \n \'documents\': [[\'Photography student Thorsten Mjölnir captures the way students decorate their laptops.\',\n                \'Why sales of very basic mobile phones, without apps and internet connection, are increasing.\',\n                "What do you do when your collection of millions of books keeps growing but your bookshelves don\'t?",\n                \'The developers of a powerful mini aircraft hope it will be used by the armed forces.\',\n                \'How tech is helping young families and couples regain their busy social lives after Covid.\',\n                \'Watch as Lee Zii Jia of Malaysia records a speed of 372km/h on his backhand point against Lakshya Sen of India in the All England Badminton Championships.\',\n                \'The Royal Mint has found a way to turn old circuit boards from phones, computers 

## Vector MAP

In [15]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [16]:

getado = collection.get(ids = "id141", 
                       include = ["documents", "embeddings"])


In [17]:
print(type(getado), getado)

<class 'dict'> {'ids': ['id141'], 'embeddings': array([[-8.08560848e-02, -4.99637052e-02, -2.37774849e-02,
        -1.10536022e-02,  2.66577117e-02, -4.47933301e-02,
        -2.88966335e-02,  2.66561043e-02,  1.43972272e-03,
        -1.64078418e-02,  6.53492734e-02, -6.90199286e-02,
        -5.74807823e-02,  1.01116151e-02,  5.04303500e-02,
        -2.05776445e-03,  7.25640804e-02, -1.24373689e-01,
         1.06594423e-02, -1.09420463e-01, -1.14324046e-02,
        -1.03760120e-02, -2.06108317e-02, -2.43940949e-02,
         7.82847628e-02,  5.82055887e-03,  2.33177263e-02,
        -8.24382976e-02, -2.72650588e-02,  4.66747722e-03,
         4.34018858e-03,  3.25280502e-02, -2.60309745e-02,
         7.96390548e-02,  4.21820618e-02, -1.21199943e-01,
         4.90708388e-02, -7.62584656e-02,  4.33162451e-02,
        -8.36045742e-02, -7.14040175e-02, -1.87925138e-02,
         3.60493883e-02,  4.28456254e-02,  2.57604383e-02,
         3.97251472e-02, -7.09129497e-03,  3.51899490e-02,
        

In [18]:
word_vectors = getado["embeddings"]
word_list = getado["documents"]

display(1, word_vectors)

display(2, word_list)

1

array([[-8.08560848e-02, -4.99637052e-02, -2.37774849e-02,
        -1.10536022e-02,  2.66577117e-02, -4.47933301e-02,
        -2.88966335e-02,  2.66561043e-02,  1.43972272e-03,
        -1.64078418e-02,  6.53492734e-02, -6.90199286e-02,
        -5.74807823e-02,  1.01116151e-02,  5.04303500e-02,
        -2.05776445e-03,  7.25640804e-02, -1.24373689e-01,
         1.06594423e-02, -1.09420463e-01, -1.14324046e-02,
        -1.03760120e-02, -2.06108317e-02, -2.43940949e-02,
         7.82847628e-02,  5.82055887e-03,  2.33177263e-02,
        -8.24382976e-02, -2.72650588e-02,  4.66747722e-03,
         4.34018858e-03,  3.25280502e-02, -2.60309745e-02,
         7.96390548e-02,  4.21820618e-02, -1.21199943e-01,
         4.90708388e-02, -7.62584656e-02,  4.33162451e-02,
        -8.36045742e-02, -7.14040175e-02, -1.87925138e-02,
         3.60493883e-02,  4.28456254e-02,  2.57604383e-02,
         3.97251472e-02, -7.09129497e-03,  3.51899490e-02,
         2.73690969e-02,  9.28983930e-03, -3.91616635e-0

2

['Acer Swift 3 featuring a 10th-generation Intel Ice Lake CPU, 2K screen, and more launched in India for INR 64999 (US$865)']

Once we have our information inside the Database we can query It, and ask for data that matches our needs. The search is done inside the content of the document, and it dosn't look for the exact word, or phrase. The results will be based on the similarity between the search terms and the content of documents. 

The metadata is not used in the search, but they can be utilized for filtering or refining the results after the initial search. 


## Loading the model and creating the prompt
TRANSFORMERS!!
Time to use the library **transformers**, the most famous library from [hugging face](https://huggingface.co/) for working with language models. 

We are importing: 
* **Autotokenizer**: It is a utility class for tokenizing text inputs that are compatible with various pre-trained language models.
* **AutoModelForCasualLLM**: it provides an interface to pre-trained language models specifically designed for language generation tasks using causal language modeling (e.g., GPT models), or the model used in this notebook ***databricks/dolly-v2-3b***.
* **pipeline**: provides a simple interface for performing various natural language processing (NLP) tasks, such as text generation (our case) or text classification. 

The model selected is [dolly-v2-3b](https://huggingface.co/databricks/dolly-v2-3b), the smallest Dolly model. It have 3billion paramaters, more than enough for our sample, and works much better than GPT2. 

Please, feel free to test [different Models](https://huggingface.co/models?pipeline_tag=text-generation&sort=trending), you need to search for NLP models trained for text-generation. My recomendation is choose "small" models, or we will run out of memory in kaggle.  


In [19]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = "databricks/dolly-v2-3b"
tokenizer = AutoTokenizer.from_pretrained(model_id)
lm_model = AutoModelForCausalLM.from_pretrained(model_id, device_map = 'cuda')



2025-01-31 15:04:32.686292: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-31 15:04:33.652327: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738350274.019995   23151 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738350274.112548   23151 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-31 15:04:34.715749: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

The next step is to initialize the pipeline using the objects created above. 

The model's response is limited to 256 tokens, for this project I'm not interested in a longer response, but it can easily be extended to whatever length you want.

Setting ***device_map*** to ***auto*** we are instructing the model to automaticaly select the most appropiate device: CPU or GPU for processing the text generation.  

In [20]:
pipe = pipeline(
    "text-generation",
    model = lm_model,
    tokenizer = tokenizer,
    max_new_tokens = 256,
    device_map = "cuda",
)

Device set to use cuda


## Creating the extended prompt
To create the prompt we use the result from query the Vector Database  and the sentence introduced by the user. 

The prompt have two parts, the **relevant context** that is the information recovered from the database and the **user's question**. 

We only need to join the two parts together to create the prompt that we are going to send to the model. 

You can limit the lenght of the context passed to the model, because we can get some Memory problems with one of the datasets that contains a realy large text in the document part. 

In [ ]:
question = "Can I buy a Toshiba laptop?"

context = " ".join([f"#{str(i)}" for i in results["documents"][0]])


prompt_template = f"Relevant context: {context}\n\n The user's question: {question}"
prompt_template

"Relevant context: #The Legendary Toshiba is Officially Done With Making Laptops #3 gaming laptop deals you can’t afford to miss today #Lenovo and HP control half of the global laptop market #Asus ROG Zephyrus G14 gaming laptop announced in India #Acer Swift 3 featuring a 10th-generation Intel Ice Lake CPU, 2K screen, and more launched in India for INR 64999 (US$865) #Apple's Next MacBook Could Be the Cheapest in Company's History #Features of Huawei's Desktop Computer Revealed #Redmi to launch its first gaming laptop on August 14: Here are all the details #Toshiba shuts the lid on laptops after 35 years #This is the cheapest Windows PC by a mile and it even has a spare SSD slot\n\n The user's question: Can I buy a Toshiba laptop?"

Now all that remains is to send the prompt to the model and wait for its response!


In [24]:
lm_response = pipe(prompt_template)

print(lm_response[0]["generated_text"])

Relevant context: #The Legendary Toshiba is Officially Done With Making Laptops #3 gaming laptop deals you can’t afford to miss today #Lenovo and HP control half of the global laptop market #Asus ROG Zephyrus G14 gaming laptop announced in India #Acer Swift 3 featuring a 10th-generation Intel Ice Lake CPU, 2K screen, and more launched in India for INR 64999 (US$865) #Apple's Next MacBook Could Be the Cheapest in Company's History #Features of Huawei's Desktop Computer Revealed #Redmi to launch its first gaming laptop on August 14: Here are all the details #Toshiba shuts the lid on laptops after 35 years #This is the cheapest Windows PC by a mile and it even has a spare SSD slot

 The user's question: Can I buy a Toshiba laptop?
The answer: No, Toshiba has decided to stop manufacturing laptops.


